In [1]:
import telepot

token = "1749792111:AAHhGbNY1zcwg76ZwIMgdkYbrae3Gfqi6xI"
mc = "1693638524"
bot = telepot.Bot(token)
bot.sendMessage(mc, "PC에서 보내는 메시지입니다.")

{'message_id': 77,
 'from': {'id': 1749792111,
  'is_bot': True,
  'first_name': 'napico',
  'username': 'napicobot'},
 'chat': {'id': 1693638524,
  'first_name': '병권',
  'last_name': '최',
  'type': 'private'},
 'date': 1617694137,
 'text': 'PC에서 보내는 메시지입니다.'}

In [ ]:
import telegram

token = "1749792111:AAHhGbNY1zcwg76ZwIMgdkYbrae3Gfqi6xI"
mc = "1693638524"
bot = telegram.Bot(token)
bot.sendMessage(mc, "안녕하세요 napico 채봇입니다.")

"""
updates = bot.getUpdates()
print(updates)
for i in updates:
    print(i)
"""

In [ ]:
# 라이브러리 불러오기
import requests
from bs4 import BeautifulSoup
import telegram
from apscheduler.schedulers.blocking import BlockingScheduler

# 검색 키워드
search_word = 'HMM'

# 텔레그램 봇 생성
token = "1749792111:AAHhGbNY1zcwg76ZwIMgdkYbrae3Gfqi6xI"
mc = "1693638524"
bot = telegram.Bot(token)

# 스케쥴러 생성
sched = BlockingScheduler()

# 기존에 보냈던 링크를 담아둘 리스트
old_links = []

# 스크래핑 함수 
def extract_links(old_links=[]):
    url = f'https://m.search.naver.com/search.naver?where=m_news&sm=mtb_jum&query={search_word}'
    req = requests.get(url)
    html = req.text
    soup = BeautifulSoup(html, 'html.parser')
    search_result = soup.select_one('#news_result_list')
    news_list = search_result.select('.bx > .news_wrap > a')

    links = []
    for news in news_list[:5]:
        link = news['href']
        links.append(link)
    
    new_links=[]
    for link in links:
        if link not in old_links:
            new_links.append(link)
    
    return new_links

"""
# 이전 링크를 매개변수로 받아서, 비교 후 새로운 링크만 출력
# 차후 이 부분을 메시지 전송 코드로 변경하고 매시간 동작하도록 설정
# 새로운 링크가 없다면 빈 리스트 반환
for i in range(3):
    new_links = extract_links(old_links)
    print('===보낼 링크===\n', new_links,'\n')
    old_links += new_links.copy()
    old_links = list(set(old_links))
"""

# 텔레그램 메시지 전송 함수
def send_links():
    global old_links
    new_links = extract_links(old_links)
    if new_links:
        for link in new_links:
            bot.sendMessage(mc, link)
    else:
        bot.sendMessage(mc, '새로운 뉴스 없음')
    old_links += new_links.copy()
    old_links = list(set(old_links))

# 최초 시작
send_links()

# 스케쥴러 세팅 및 작동
sched.add_job(send_links, 'interval', minutes=5)
sched.start()
